# Assignment 4: Wheres Waldo?
### Name: Eileanor LaRocco
In this assignment, you will develop an object detection algorithm to locate Waldo in a set of images. You will develop a model to detect the bounding box around Waldo. Your final task is to submit your predictions on Kaggle for evaluation.

### Imports

In [1]:
import os
import opendatasets as od
import pandas as pd
import numpy as np
import random
import csv
import matplotlib.pyplot as plt

import shutil
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image

In [2]:
SEED = 1

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = device = torch.device("cuda") #mps/cuda
print(device)

cuda


### Paths

In [3]:
train_folder = "2024-fall-ml-3-hw-4-wheres-waldo/train/train" # Original Train Images
test_folder = "2024-fall-ml-3-hw-4-wheres-waldo/test/test" # Original Test Images
annotations_file = "2024-fall-ml-3-hw-4-wheres-waldo/annotations.csv" # Original Annotations File
image_sz = 512

In [4]:
# Define the dataset (Train and Test Loaders)
class WaldoDataset(torch.utils.data.Dataset):
    def __init__(self, annotations_file, img_dir, transforms=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        #image = torchvision.transforms.ToTensor()(image)  # Convert to tensor
        
        box_data = self.img_labels.iloc[idx, 1:].values
        boxes = [float(item) for item in box_data]
        
        boxes = torch.as_tensor([boxes], dtype=torch.float32)
        labels = torch.ones((1,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((1,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            image = self.transforms(image)

        return image, target


# Set up the dataset and data loaders
train_dataset = WaldoDataset(
    annotations_file="2024-fall-ml-3-hw-4-wheres-waldo/train_annotations.csv",
    img_dir="2024-fall-ml-3-hw-4-wheres-waldo/train/chunks",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((image_sz, image_sz)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
)

val_dataset = WaldoDataset(
    annotations_file="2024-fall-ml-3-hw-4-wheres-waldo/test_annotations.csv",
    img_dir="2024-fall-ml-3-hw-4-wheres-waldo/train/val",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((image_sz, image_sz)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
)

train_data_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=lambda x: list(zip(*x))
)

val_data_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    collate_fn=lambda x: list(zip(*x))
)


# Model

### Architecture

### Training

In [6]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.ops import box_iou

# Define IoU computation
def calculate_iou(pred_boxes, target_boxes):
    """
    Computes IoU between predicted and target boxes.
    Args:
        pred_boxes (Tensor): Predicted boxes, shape (N, 4).
        target_boxes (Tensor): Target boxes, shape (M, 4).
    Returns:
        IoU scores: Tensor of shape (N, M).
    """
    return ops.box_iou(pred_boxes, target_boxes)

# Training function
def train_one_epoch(model, data_loader, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, targets in data_loader:
        images = torch.stack([img.to(device) for img in images])
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()

        # Forward pass
        losses = model(images, targets)

        # Ensure that losses is a dictionary and sum all loss components
        if isinstance(losses, dict):
            total_loss = sum(loss for loss in losses.values())
        else:
            raise ValueError("Expected losses to be a dictionary")

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()

    epoch_loss = running_loss / len(data_loader)
    return epoch_loss



# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_iou = 0.0
    num_samples = 0

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Validation", leave=False):
            # Move data to device
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Generate predictions
            outputs = model(images)  # During evaluation, model outputs predictions
            
            # Compute IoU between predicted and target boxes
            for output, target in zip(outputs, targets):
                pred_boxes = output["boxes"].detach()
                target_boxes = target["boxes"]

                if len(pred_boxes) > 0 and len(target_boxes) > 0:
                    iou = box_iou(pred_boxes, target_boxes).mean().item()
                    total_iou += iou
                else:
                    total_iou += 0  # No predictions or no targets

                num_samples += 1

    # Calculate average IoU
    avg_iou = total_iou / num_samples if num_samples > 0 else 0.0

    return avg_iou


# Main training loop
def train_model(model, train_loader, val_loader, num_epochs, device, optimizer, scheduler, checkpoint_path):
    best_iou = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Train the model
        train_loss = train_one_epoch(model, train_loader, optimizer, device)

        # Validate the model
        val_iou = evaluate(model, val_loader, device)

        # Save the best model
        if val_iou > best_iou:
            best_iou = val_iou
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model saved with IoU: {best_iou:.4f}")

        # Update the learning rate
        scheduler.step()

        print(f"Training Loss: {train_loss:.4f}, Validation IoU: {val_iou:.4f}")

    print(f"Training complete. Best Validation IoU: {best_iou:.4f}")


# Set up optimizer, scheduler, and device
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
checkpoint_path = "best_model.pth"
num_epochs = 10

# Train and evaluate the model
train_model(
    model,
    train_data_loader,
    val_data_loader,
    num_epochs,
    device,
    optimizer,
    scheduler,
    checkpoint_path
)


Epoch 1/10


OutOfMemoryError: CUDA out of memory. Tried to allocate 626.00 MiB. GPU 0 has a total capacity of 44.34 GiB of which 213.75 MiB is free. Process 901941 has 12.85 GiB memory in use. Process 903143 has 14.31 GiB memory in use. Including non-PyTorch memory, this process has 16.84 GiB memory in use. Of the allocated memory 15.72 GiB is allocated by PyTorch, and 813.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Submission File 

In [ ]:
predictions = []

output_directory = "2024-fall-ml-3-hw-4-wheres-waldo/outputs"
if not os.path.exists(output_directory):
        os.makedirs(output_directory)

# Function to predict bounding box on a new image
# Function to predict bounding box on a new image
# Function to predict bounding box on a new image
def predict(model, image_path, transform=None):
    model.eval()  # Set the model to evaluation mode

    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")  # Open image and convert to RGB
    orig_width, orig_height = image.size  # Get original image dimensions
    if transform:
        image = transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension

    # Ensure image is a Tensor and move it to the correct device
    image = image.to(device)

    # Predict the bounding box
    with torch.no_grad():
        # Get the model's output (a list of dictionaries in this case)
        outputs = model(image)
        if isinstance(outputs, list):  # Check if the output is a list
            outputs = outputs[0]  # Get the first dictionary in the list
        
        # Check if any bounding boxes were predicted
        if len(outputs['boxes']) == 0:
            print(f"No objects detected in {image_path}")
            return None  # No detection, return None or handle as needed
        
        # Extract the predicted bounding box (assuming the first detected object)
        predicted_bbox = outputs['boxes'][0].cpu().numpy()  # Assuming 'boxes' contains the bounding boxes

    # Scale bbox back to original image dimensions
    predicted_bbox[0] *= orig_width / image_sz  # x_min
    predicted_bbox[1] *= orig_height / image_sz  # y_min
    predicted_bbox[2] *= orig_width / image_sz  # x_max
    predicted_bbox[3] *= orig_height / image_sz  # y_max

    return predicted_bbox


# Define image transformations (resize, normalization)
transform = transforms.Compose([
    transforms.Resize((image_sz, image_sz)),  # Resize the image to 512x512
    transforms.ToTensor(),  # Convert the image to a tensor
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#For each image in test folder, predict, draw bounding box, save image, and save into csv file for submission
test_images = [img for img in os.listdir(test_folder) if img.endswith(".jpg")]
predictions = []

for name in test_images:
    image_path = os.path.join(test_folder, name)  # Replace with the path to your test image
    predicted_bbox = predict(model, image_path, transform)
    
    if predicted_bbox is None:
        continue  # Skip the image if no bounding box is detected

    # Print the predicted bounding box (x_min, y_min, x_max, y_max)
    print("Predicted Bounding Box:", predicted_bbox)

    # Plot the image and the predicted bounding box
    image = Image.open(image_path)
    plt.imshow(image)
    
    # Add the bounding box only if a prediction was made
    plt.gca().add_patch(plt.Rectangle(
        (predicted_bbox[0], predicted_bbox[1]),  # (x_min, y_min)
        predicted_bbox[2] - predicted_bbox[0],  # Width (x_max - x_min)
        predicted_bbox[3] - predicted_bbox[1],  # Height (y_max - y_min)
        linewidth=2, edgecolor='r', facecolor='none'
    ))
    
    plt.xticks([])
    plt.yticks([])
    plt.savefig(os.path.join(output_directory, name), bbox_inches='tight')
    plt.show()

    # Extract bounding box coordinates
    x_min, y_min, x_max, y_max = predicted_bbox
    predictions.append([name, x_min, y_min, x_max, y_max])

# Save predictions to CSV
df = pd.DataFrame(predictions, columns=["filename", "xmin", "ymin", "xmax", "ymax"])
df.to_csv(os.path.join(output_directory, 'predictions.csv'), index=False)
